In [ ]:
import findspark
#findspark.init(spark_path)
findspark.init()

from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkContext, SparkConf

In [ ]:
conf = SparkConf().setAppName('log_analysis').setMaster('local')
sc = SparkContext(conf=conf)
spark = SQLContext(sc)